<a href="https://colab.research.google.com/gist/ds-semenov/54360e66c37d33eaf0a4bbabc08795a4/sib200_ru_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets transformers evaluate optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 19.8 MB/s eta 0:00:00


In [2]:
pass

In [3]:
pass

In [4]:
from typing import List, Tuple

In [5]:
from datasets import load_dataset, Dataset, concatenate_datasets
import evaluate
import numpy as np
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
import optuna
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer, set_seed
from transformers import pipeline

In [6]:
MODEL_NAME = "ai-forever/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
set_seed(42)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [7]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md: 0.00B [00:00, ?B/s]

train.tsv: 0.00B [00:00, ?B/s]

dev.tsv: 0.00B [00:00, ?B/s]

test.tsv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
print(train_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})


In [9]:
MAX_LEN = 192
MINIBATCH_SIZE = 64

tokenized_train_set = train_set.map(
    lambda it: tokenizer(it["text"], truncation=True, max_length=MAX_LEN),
    batched=True, batch_size=MINIBATCH_SIZE, remove_columns=["text"]
)
tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it["text"], truncation=True, max_length=MAX_LEN),
    batched=True, batch_size=MINIBATCH_SIZE, remove_columns=["text"]
)
tokenized_test_set = test_set.map(
    lambda it: tokenizer(it["text"], truncation=True, max_length=MAX_LEN),
    batched=True, batch_size=MINIBATCH_SIZE, remove_columns=["text"]
)

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [10]:
print(tokenized_train_set)

Dataset({
    features: ['index_id', 'category', 'input_ids', 'attention_mask'],
    num_rows: 701
})


In [11]:
cls_metric = evaluate.load('f1')

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [15]:
labeled_train_set = tokenized_train_set.add_column(
    "label",
    [label2id[val] for val in tokenized_train_set["category"]]
).remove_columns(["category"])

labeled_validation_set = tokenized_validation_set.add_column(
    "label",
    [label2id[val] for val in tokenized_validation_set["category"]]
).remove_columns(["category"])

labeled_test_set = tokenized_test_set.add_column(
    "label",
    [label2id[val] for val in tokenized_test_set["category"]]
).remove_columns(["category"])


In [16]:
print(labeled_train_set)

Dataset({
    features: ['index_id', 'input_ids', 'attention_mask', 'label'],
    num_rows: 701
})


In [17]:
BEST_MODEL_HP = {}

def model_init(trial=None):
    if trial is None:
        hid_d = BEST_MODEL_HP.get("hidden_dropout_prob", 0.1)
        attn_d = BEST_MODEL_HP.get("attention_probs_dropout_prob", 0.1)
        cls_d = BEST_MODEL_HP.get("classifier_dropout", 0.1)
    else:
        hid_d = trial.suggest_float("hidden_dropout_prob", 0.05, 0.25)
        attn_d = trial.suggest_float("attention_probs_dropout_prob", 0.05, 0.25)
        cls_d = trial.suggest_float("classifier_dropout", 0.0, 0.3)

    config = AutoConfig.from_pretrained(
        MODEL_NAME,
        num_labels=n_categories,
        id2label=id2label,
        label2id=label2id,
    )
    config.hidden_dropout_prob = hid_d
    config.attention_probs_dropout_prob = attn_d
    config.classifier_dropout = cls_d

    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

class_weights = None
sampler = None

def set_balancing(train_ds):
    global class_weights, sampler
    y = np.array(train_ds["label"])
    classes = np.unique(y)
    cw = compute_class_weight(class_weight="balanced", classes=classes, y=y)
    class_weights = torch.tensor(cw, dtype=torch.float)
    SAMPLE_ALPHA = 0.5
    sample_w = (class_weights[y] ** SAMPLE_ALPHA)
    sampler = WeightedRandomSampler(weights=sample_w, num_samples=len(sample_w), replacement=True)
class BalancedTrainer(Trainer):
    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            sampler=sampler,
            collate_fn=self.data_collator,
            drop_last=False,
        )

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels", inputs.get("label"))
        model_inputs = {k: v for k, v in inputs.items() if k in ["input_ids", "attention_mask", "token_type_ids"]}
        outputs = model(**model_inputs)
        w = class_weights.to(outputs.logits.device)
        loss = F.cross_entropy(
            outputs.logits,
            labels,
            weight=w,
            label_smoothing=float(getattr(self.args, "label_smoothing_factor", 0.0)),
        )
        return (loss, outputs) if return_outputs else loss

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 2.5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.06),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.12),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.0, 0.15),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 6),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [4, 8, 16]),
    }

search_args = TrainingArguments(
    output_dir="hp_search",
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="epoch",
    report_to="none",
    fp16=True,
    gradient_checkpointing=True,
    seed=42,
    data_seed=42,
)

set_seed(42)
set_balancing(labeled_train_set)

trainer = BalancedTrainer(
    model_init=model_init,
    args=search_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

best = trainer.hyperparameter_search(
    backend="optuna",
    direction="maximize",
    n_trials=80,
    hp_space=hp_space,
    compute_objective=lambda m: m["eval_f1"],
)

best_hp = dict(best.hyperparameters)

drop_keys = ["hidden_dropout_prob", "attention_probs_dropout_prob", "classifier_dropout"]
BEST_MODEL_HP = {k: best_hp.pop(k) for k in drop_keys if k in best_hp}

best_hp["per_device_eval_batch_size"] = best_hp["per_device_train_batch_size"]

full_train = concatenate_datasets([labeled_train_set, labeled_validation_set])
set_balancing(full_train)

def predict_logits(model, ds):
    model.eval()
    device = next(model.parameters()).device
    dl = DataLoader(ds, batch_size=32, collate_fn=data_collator, drop_last=False)
    outs = []
    for batch in dl:
        batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask", "token_type_ids"]}
        with torch.no_grad():
            outs.append(model(**batch).logits.detach().cpu())
    return torch.cat(outs, dim=0)


seeds = [13, 21, 42, 77, 101, 123, 202]
logits_val = None
logits_test = None

for s in seeds:
    set_seed(s)
    args_s = TrainingArguments(
        output_dir=f"final_full_{s}",
        eval_strategy="no",
        save_strategy="no",
        logging_strategy="no",
        report_to="none",
        fp16=True,
        lr_scheduler_type="cosine",
        seed=s,
        data_seed=s,
        **best_hp,
    )
    tr = BalancedTrainer(
        model_init=model_init,
        args=args_s,
        train_dataset=full_train,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    tr.train()
    m = tr.model.cuda()
    lv = predict_logits(m, labeled_validation_set)
    lt = predict_logits(m, labeled_test_set)
    logits_val = lv if logits_val is None else logits_val + lv
    logits_test = lt if logits_test is None else logits_test + lt
    del m, tr
    torch.cuda.empty_cache()

logits_val = logits_val / len(seeds)
logits_test = logits_test / len(seeds)

pred_val = logits_val.argmax(dim=1).numpy().tolist()
pred_test = logits_test.argmax(dim=1).numpy().tolist()

y_pred_val = [id2label[i] for i in pred_val]
y_true_val = [id2label[i] for i in list(labeled_validation_set["label"])]
print(classification_report(y_true=y_true_val, y_pred=y_pred_val))

y_pred_test = [id2label[i] for i in pred_test]
y_true_test = [id2label[i] for i in list(labeled_test_set["label"])]
print(classification_report(y_true=y_true_test, y_pred=y_pred_test))


pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-12-24 05:00:05,309] A new study created in memory with name: no-name-8e31901b-1469-46ab-ae22-cfdea1f79457
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,1.890600,1.627137,0.497184
2,1.230500,0.909500,0.780722
3,0.765800,0.809415,0.793152
4,0.640600,0.802673,0.820985


[I 2025-12-24 05:04:47,929] Trial 0 finished with value: 0.8209848006399731 and parameters: {'learning_rate': 8.353511893335669e-06, 'weight_decay': 0.0020551350738586315, 'warmup_ratio': 0.0820948318868611, 'label_smoothing_factor': 0.08024558466615211, 'num_train_epochs': 4, 'per_device_train_batch_size': 2, 'gradient_accumulation_steps': 8, 'hidden_dropout_prob': 0.1681115315063974, 'attention_probs_dropout_prob': 0.23712756186445133, 'classifier_dropout': 0.033282504102332286}. Best is trial 0 with value: 0.8209848006399731.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


                    precision    recall  f1-score   support

     entertainment       0.86      0.67      0.75         9
         geography       0.73      1.00      0.84         8
            health       0.92      1.00      0.96        11
          politics       1.00      1.00      1.00        14
science/technology       1.00      0.92      0.96        25
            sports       0.92      1.00      0.96        12
            travel       0.89      0.85      0.87        20

          accuracy                           0.92        99
         macro avg       0.90      0.92      0.91        99
      weighted avg       0.93      0.92      0.92        99

                    precision    recall  f1-score   support

     entertainment       0.88      0.74      0.80        19
         geography       0.80      0.94      0.86        17
            health       0.91      0.95      0.93        22
          politics       1.00      0.97      0.98        30
science/technology       0.94      0